In [ ]:
import os
import re
import numpy as np
import matplotlib.image as mpimg
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
import glob
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from PIL import Image
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import cv2 as cv
from keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [ ]:
input_data="chest_Xray/"
train_directory= input_data + "train/"
val_directory= input_data + "val/"
test_directory= input_data + "test/"

data_dir = train_directory
class_names=os.listdir(train_directory)
print(class_names)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_model_accuracy(train_folder, validation_folder, batch_size, image_size, optimizer, epochs):
    # print("Evaluating " + param_evaluated)

    # Prepare iterators
    datagen = ImageDataGenerator(rescale=1.0 / 255.0)
    train_it = datagen.flow_from_directory(directory=train_folder,
                                           class_mode='binary', batch_size=batch_size,
                                           target_size=(image_size, image_size))
    val_it = datagen.flow_from_directory(directory=validation_folder,
                                         class_mode='binary', batch_size=batch_size,
                                         target_size=(image_size, image_size))
    test_it = datagen.flow_from_directory(test_directory,
                                          class_mode='binary', batch_size=batch_size,
                                          target_size=(image_size, image_size))

    # create model
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(image_size, image_size, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Define learning rate scheduler
    def lr_scheduler(epoch):
        lr = 0.001
        if epoch > 10:
            lr = 0.0001
        return lr

    # Create learning rate scheduler callback
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    # Train the model
    model.fit(train_it,
              validation_data=val_it,
              steps_per_epoch=train_it.n // train_it.batch_size,
              validation_steps=val_it.n // val_it.batch_size,
              epochs=epochs, verbose=1,
              callbacks=[lr_scheduler_callback])

    # Evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
    return acc

In [131]:
def get_model(train_folder, validation_folder, batch_size, image_size, optimizer, epochs):
    # Prepare iterators
    datagen = ImageDataGenerator(rescale=1.0 / 255.0)
    train_it = datagen.flow_from_directory(directory=train_folder,
                                           class_mode='binary', batch_size=batch_size,
                                           target_size=(image_size, image_size))
    val_it = datagen.flow_from_directory(directory=validation_folder,
                                         class_mode='binary', batch_size=batch_size,
                                         target_size=(image_size, image_size))
    test_it = datagen.flow_from_directory(test_directory,
                                          class_mode='binary', batch_size=batch_size,
                                          target_size=(image_size, image_size))

    # create model
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(image_size, image_size, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Define learning rate scheduler
    def lr_scheduler(epoch):
        lr = 0.001
        if epoch > 10:
            lr = 0.0001
        return lr

    # Create learning rate scheduler callback
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    # Train the model
    history = model.fit(train_it,
              validation_data=val_it,
              steps_per_epoch=train_it.n // train_it.batch_size,
              validation_steps=val_it.n // val_it.batch_size,
              epochs=epochs, verbose=1,
              callbacks=[lr_scheduler_callback])

    return model, history

In [128]:
BATCH_SIZE_128 = 128
BATCH_SIZE_64 = 64
BATCH_SIZE_32 = 32
batch_size_array = [BATCH_SIZE_128, BATCH_SIZE_64, BATCH_SIZE_32]

OPTIMIZER_ADAM = 'adam'
OPTIMIZER_SGD = 'sgd'
OPTIMIZER_RMSPROP= 'rmsprop'
optimizer_array = [OPTIMIZER_ADAM, OPTIMIZER_SGD, OPTIMIZER_RMSPROP]

EPOCHS_5 = 5
EPOCHS_10 = 10
EPOCHS_20 = 20
epoch_array = [EPOCHS_5, EPOCHS_10, EPOCHS_20]

# Common image sizes include 64x64, 128x128, 28x28 (MNIST), and 224x224 (VGG-16).
IMAGE_SIZE_128 = 128
IMAGE_SIZE_64 = 64
IMAGE_SIZE_28 = 28
image_size_array = [IMAGE_SIZE_128, IMAGE_SIZE_64, IMAGE_SIZE_28]

In [ ]:
# Define the parameter grid
param_grid = {
    'batch_size': batch_size_array,
    'image_size': image_size_array,
    'optimizer': optimizer_array,
    'epochs': epoch_array
}


# Perform grid search
best_acc = 0.0
best_params = {}
for batch_size in param_grid['batch_size']:
    print("Evaluating batch_size:", batch_size)
    for image_size in param_grid['image_size']:
        print("Evaluating image_size:", image_size)
        for optimizer in param_grid['optimizer']:
            print("Evaluating optimizer:", optimizer)
            for epochs in param_grid['epochs']:
                print("Evaluating epochs:", epochs)
                acc = get_model_accuracy(train_directory, val_directory, batch_size, image_size,
                                         optimizer, epochs)
                if acc > best_acc:
                    best_acc = acc
                    best_params = {'batch_size': batch_size, 'image_size': image_size,
                                   'optimizer': optimizer, 'epochs': epochs}

print("Best parameters:", best_params)
print("Best accuracy:", best_acc)

In [129]:
print("Best parameters:", best_params)
print("Best accuracy:", best_acc)

# Best parameters: {'batch_size': 64, 'image_size': 128, 'optimizer': 'sgd', 'epochs': 5}
# Best accuracy: 0.8301281929016113

Best parameters: {'batch_size': 64, 'image_size': 128, 'optimizer': 'sgd', 'epochs': 5}
Best accuracy: 0.8301281929016113


In [132]:
best_model, history = get_model(train_directory, val_directory, best_params['batch_size'], best_params['image_size'], best_params['optimizer'], best_params['epochs'])

Found 5216 images belonging to 2 classes.
Found 17 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/5
81/81 [==============================] - 37s 450ms/step - loss: 0.4843 - accuracy: 0.7727 - lr: 0.0010
Epoch 2/5
81/81 [==============================] - 37s 459ms/step - loss: 0.3337 - accuracy: 0.8736 - lr: 0.0010
Epoch 3/5
81/81 [==============================] - 36s 450ms/step - loss: 0.2741 - accuracy: 0.9028 - lr: 0.0010
Epoch 4/5
81/81 [==============================] - 36s 440ms/step - loss: 0.2398 - accuracy: 0.9185 - lr: 0.0010
Epoch 5/5
81/81 [==============================] - 35s 437ms/step - loss: 0.2098 - accuracy: 0.9299 - lr: 0.0010


In [133]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_it = datagen.flow_from_directory(test_directory,
                                          class_mode='binary', batch_size=best_params['batch_size'],
                                          target_size=(best_params['image_size'], best_params['image_size']))

_, accuracy = best_model.evaluate(test_it, steps=len(test_it), verbose=1)

print('Accuracy on test set: %.3f' % (accuracy * 100.0))

Found 624 images belonging to 2 classes.
10/10 [==============================] - 3s 304ms/step - loss: 0.3603 - accuracy: 0.8317
Accuracy on test set: 83.173
